In [3]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")


In [12]:
from penzai import pz
import json

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector


In [4]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

RuntimeError: Unable to initialize backend 'tpu': ABORTED: The TPU is already in use by process with pid 1484045. Not attempting to load libtpu.so in this process. (set JAX_PLATFORMS='' to automatically choose an available backend)

In [ ]:
from transformers import AutoTokenizer
import jax
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [10]:
from sprint.task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

In [4]:
import json, numpy as np

with open("cleanup_results_gemma_2_algo.jsonl", "r") as f:
    cleanup_results = [json.loads(line) for line in f]


with open("cleanup_results_gemma_2_all.jsonl", "r") as f:
    tmp = [json.loads(line) for line in f]
    tmp = [x for x in tmp if not x["task"].startswith("algo")]

    cleanup_results += tmp

In [6]:
task = "algo_last"
layers = [14, 16, 18, 20, 22]

task_results = [result for result in cleanup_results if result["layer"] in layers]

print(len(task_results))

95


In [7]:
from micrlhf.utils.load_sae import sae_encode, get_dm_res_sae

thresholds = {
    layer: get_dm_res_sae(layer).get("threshold", 0) for layer in layers
}


In [8]:
with open("gemma_2_cleaning_compact.jsonl", "w") as f:
    for r in task_results:
        task = r["task"]
        layer = r["layer"]
        
        weights = np.array(r["weights"])

        i = np.argwhere(weights > thresholds[layer]).flatten()
        w = weights[i]

        idx = np.argsort(w)[::-1]

        i = i[idx]
        w = w[idx]

        data = {
            "task": task,
            "weights": w.tolist(),
            "indices": i.tolist(),
            "tv loss": r["tv_loss"],
            "cleaning loss": r["loss"],
            "ito loss": r["ito_loss"],
            "layer": layer
        }

        print(data)
        f.write(json.dumps(data) + "\n")

{'task': 'algo_last', 'weights': [56.39113998413086, 42.13969039916992, 30.260478973388672, 14.489788055419922, 5.450676918029785], 'indices': [11265, 6304, 5088, 8060, 6247], 'tv loss': 1.921875, 'cleaning loss': 0.2197265625, 'ito loss': 2.0, 'layer': 14}
{'task': 'algo_last', 'weights': [50.45500564575195, 45.987667083740234, 40.501956939697266, 22.649768829345703], 'indices': [5637, 1160, 9343, 3969], 'tv loss': 0.99609375, 'cleaning loss': 0.2060546875, 'ito loss': 0.55859375, 'layer': 16}
{'task': 'algo_last', 'weights': [47.48284149169922, 40.43308639526367, 33.862640380859375, 31.790987014770508], 'indices': [11687, 1255, 14456, 14772], 'tv loss': 2.15625, 'cleaning loss': 0.234375, 'ito loss': 0.4375, 'layer': 18}
{'task': 'algo_last', 'weights': [64.82637786865234, 53.81707000732422, 47.99002456665039, 44.49906921386719, 38.31149673461914, 28.91067886352539, 20.28519058227539, 12.863045692443848, 8.231931686401367], 'indices': [5081, 5328, 7445, 9356, 12492, 6814, 2977, 16206